In [97]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm

from pangres import upsert
from sqlalchemy import text, create_engine

from concurrent.futures import as_completed, ProcessPoolExecutor
from requests_futures.sessions import FuturesSession
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
from datetime import datetime

load_dotenv()

True

In [2]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
db_name=os.environ.get("db_name")


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

In [ ]:
def retrieve_user_match_ids(UHE_varsity_2025_2026: dict[str, str], api_key:str, type:str='ranked', start:int=0, count:int=100):
    match_id_session = FuturesSession(executor=ProcessPoolExecutor(max_workers=10))
    retries = 5
    status_forcelist = [429]
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        respect_retry_after_header=True,
        status_forcelist=status_forcelist,
    )

    adapter = HTTPAdapter(max_retries=retry)

    match_id_session.mount('http://', adapter)
    match_id_session.mount('https://', adapter)

    # Retrieve match data from match_ids
    # would be a good idea to track which match_ids belong to which user
    puuids = [user['puuid'] for user in UHE_varsity_2025_2026]
    match_id_threads = [match_id_session.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type={type}&start={start}&count={count}&api_key={api_key}') for puuid in puuids]

    match_id_to_puuid_list = []
    t1 = time.time()
    for future in tqdm(as_completed(match_id_threads)):
        resp = future.result()
        match_ids = resp.json()
        puuid = puuids[match_id_threads.index(future)]
        username = UHE_varsity_2025_2026[match_id_threads.index(future)]['username']
        match_id_to_puuid_dict = {
            'username': username,
            'puuid': puuid,
            'match_ids': match_ids
        }
        match_id_to_puuid_list.append(match_id_to_puuid_dict)
    t2 = time.time()
    print(f'EXTRACTING MATCH IDS -- {round(t2 - t1, 2)}s')
    return match_id_to_puuid_list


In [ ]:
def process_match_json(game, username, puuid, type='ranked'):

    side_dict = {
        100:'blue',
        200:'red'
    }

    metadata = game['metadata']
    data_version = metadata['dataVersion']
    match_id = metadata['matchId']
    participants_puuid = metadata['participants']

    match_info = game['info']
    game_start_timestamp_in_unix_milliseconds = match_info['gameStartTimestamp']
    game_end_timestamp_in_unix_milliseconds = match_info['gameEndTimestamp']
    game_start_datetime = datetime.fromtimestamp(game_start_timestamp_in_unix_milliseconds / 1000)
    game_end_datetime = datetime.fromtimestamp(game_end_timestamp_in_unix_milliseconds / 1000)
    match_duration_in_seconds = match_info['gameDuration'] # 
    match_mode = match_info['gameMode']
    match_type = match_info['gameType']
    queue_type = type
    
    match_version = match_info['gameVersion']
    match_map_id = match_info['mapId']
    match_queue_id = match_info['queueId']
    participant_data = match_info['participants']

    main_player = participant_data[participants_puuid.index(puuid)]
    champion_level = main_player['champLevel']
    champion_experience = main_player['champExperience']
    champion_name = main_player['championName']
    champion_id = main_player['championId']
    deaths = main_player['deaths']
    kills = main_player['kills']
    assists = main_player['assists']
    gold_earned = main_player['goldEarned']
    gold_spent = main_player['goldSpent']
    individual_position = main_player['individualPosition']
    item_0 = main_player['item0']
    item_1 = main_player['item1']
    item_2 = main_player['item2']
    item_3 = main_player['item3']
    item_4 = main_player['item4']
    item_5 = main_player['item5']
    item_6 = main_player['item6']
    lane = main_player['lane']
    neutral_minions_killed = main_player['neutralMinionsKilled']
    objectives_stolen = main_player['objectivesStolen']
    participant_id = main_player['participantId']
    riot_id_game_name = main_player['riotIdGameName']
    riot_id_tagline = main_player['riotIdTagline']
    role = main_player['role']
    total_damage_dealt = main_player['totalDamageDealt']
    total_damage_dealt_to_champions = main_player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = main_player['totalDamageShieldedOnTeammates']
    total_damage_taken = main_player['totalDamageTaken']
    totalHealsOnTeammates = main_player['totalHealsOnTeammates']
    total_minions_killed = main_player['totalMinionsKilled']
    total_time_cc_dealt = main_player['totalTimeCCDealt']
    team_id = side_dict[main_player['teamId']]
    team_position = main_player['teamPosition']
    turret_kills = main_player['turretKills']
    vision_score = main_player['visionScore']
    vision_wards_bought = main_player['visionWardsBoughtInGame']
    wards_placed = main_player['wardsPlaced']
    wards_killed = main_player['wardsKilled']
    win = main_player['win']

    # perks
    main_player_perks = main_player['perks']
    stats_perks = main_player_perks['statPerks']
    perk_defense = stats_perks['defense']
    perk_flex = stats_perks['flex']
    perk_offense = stats_perks['offense']

    perk_style_selection = main_player_perks['styles']

    primary_style = perk_style_selection[0]
    primary_style_selections = primary_style['selections']
    keystone = primary_style_selections[0]['perk']
    primary_style_selection_1 = primary_style_selections[1]['perk']
    primary_style_selection_2 = primary_style_selections[2]['perk']
    primary_style_selection_3 = primary_style_selections[3]['perk']

    secondary_style = perk_style_selection[1]
    secondary_style_selections = secondary_style['selections']
    secondary_style_selection_1 = secondary_style_selections[0]['perk']
    secondary_style_selection_2 = secondary_style_selections[1]['perk']

    baron_kills = main_player['baronKills']
    dragon_kills = main_player['dragonKills']
    firstBlood = main_player['firstBloodKill']
    firstBloodAssist = main_player['firstBloodAssist']
    firstTower = main_player['firstTowerKill']
    firstTowerAssist = main_player['firstTowerAssist']

    teams = match_info['teams']
    for team in teams:
        if team['teamId'] == main_player['teamId']:
            for key, value in team['objectives'].items():
                if key == 'atakhan':
                    team_atakhan_kill_first = value['first']
                    team_atakhan_kills = value['kills']
                elif key == 'baron':
                    team_baron_kill_first = value['first']
                    team_baron_kills = value['kills']
                elif key == 'champion':
                    team_champion_kill_first = value['first']
                    team_champion_kills = value['kills']
                elif key == 'dragon':
                    team_dragon_kill_first = value['first']
                    team_dragon_kills = value['kills']
                elif key == 'horde':
                    team_horde_kill_first = value['first']
                    team_horde_kills = value['kills']
                elif key == 'inhibitor':
                    team_inhibitor_kill_first = value['first']
                    team_inhibitor_kills = value['kills']
                elif key == 'riftHerald':  
                    team_rift_herald_kill_first = value['first']
                    team_rift_herald_kills = value['kills']
                elif key == 'tower':
                    team_tower_kill_first = value['first']
                    team_tower_kills = value['kills']
        else:
            for key, value in team['objectives'].items():
                if key == 'atakhan':
                    opposing_team_atakhan_kill_first = value['first']
                    opposing_team_atakhan_kills = value['kills']
                elif key == 'baron':
                    opposing_team_baron_kill_first = value['first']
                    opposing_team_baron_kills = value['kills']
                elif key == 'champion':
                    opposing_team_champion_kill_first = value['first']
                    opposing_team_champion_kills = value['kills']
                elif key == 'dragon':
                    opposing_team_dragon_kill_first = value['first']
                    opposing_team_dragon_kills = value['kills']
                elif key == 'horde':
                    opposing_team_horde_kill_first = value['first']
                    opposing_team_horde_kills = value['kills']
                elif key == 'inhibitor':
                    opposing_team_inhibitor_kill_first = value['first']
                    opposing_team_inhibitor_kills = value['kills']
                elif key == 'riftHerald':
                    opposing_team_rift_herald_kill_first = value['first']
                    opposing_team_rift_herald_kills = value['kills']
                elif key == 'tower':
                    opposing_team_tower_kill_first = value['first']
                    opposing_team_tower_kills = value['kills']

    match_dataframe = pd.DataFrame(
        {
            'data_version': [data_version],
            'match_id': [match_id],
            'puuid': [puuid],
            'participants_puuid': [participants_puuid],
            'game_start_timestamp_in_unix_milliseconds': [game_start_timestamp_in_unix_milliseconds],
            'game_end_timestamp_in_unix_milliseconds': [game_end_timestamp_in_unix_milliseconds],
            'game_start_datetime': [game_start_datetime],
            'game_end_datetime': [game_end_datetime],
            'match_mode': [match_mode],
            'match_type': [match_type],
            'queue_type': [queue_type],
            'match_duration_in_seconds': [match_duration_in_seconds],
            'match_version': [match_version],
            'match_map_id': [match_map_id],
            'match_queue_id': [match_queue_id],
            'champion_level': [champion_level],
            'champion_experience': [champion_experience],
            'champion_name': [champion_name],
            'champion_id': [champion_id],
            'deaths': [deaths],
            'kills': [kills],
            'assists': [assists],
            'gold_earned': [gold_earned],
            'gold_spent': [gold_spent],
            'individual_position': [individual_position],
            'item_0': [item_0],
            'item_1': [item_1],
            'item_2': [item_2],
            'item_3': [item_3],
            'item_4': [item_4],
            'item_5': [item_5],
            'item_6': [item_6],
            'lane': [lane],
            'neutral_minions_killed': [neutral_minions_killed],
            'objectives_stolen': [objectives_stolen],
            'participant_id': [participant_id],
            'riot_id_game_name': [riot_id_game_name],
            'riot_id_tagline': [riot_id_tagline],
            'role': [role],
            'total_damage_dealt': [total_damage_dealt],
            'total_damage_dealt_to_champions': [total_damage_dealt_to_champions],
            'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
            'total_damage_taken': [total_damage_taken],
            'totalHealsOnTeammates': [totalHealsOnTeammates],
            'total_minions_killed': [total_minions_killed],
            'total_time_cc_dealt': [total_time_cc_dealt],
            'team_id': [team_id],
            'team_position': [team_position],
            'turret_kills': [turret_kills],
            'vision_score': [vision_score],
            'vision_wards_bought': [vision_wards_bought],
            'wards_placed': [wards_placed],
            'wards_killed': [wards_killed],
            'win': [win],
            'keystone': [keystone],
            'primary_style_selection_1': [primary_style_selection_1],
            'primary_style_selection_2': [primary_style_selection_2],
            'primary_style_selection_3': [primary_style_selection_3],
            'secondary_style_selection_1': [secondary_style_selection_1],
            'secondary_style_selection_2': [secondary_style_selection_2],
            'perk_defense': [perk_defense],
            'perk_flex': [perk_flex],
            'perk_offense': [perk_offense],
            'baron_kills': [baron_kills],
            'dragon_kills': [dragon_kills],
            'firstBlood': [firstBlood],
            'firstBloodAssist': [firstBloodAssist],
            'firstTower': [firstTower],
            'firstTowerAssist': [firstTowerAssist],
            'team_atakhan_kill_first': [team_atakhan_kill_first],
            'team_atakhan_kills': [team_atakhan_kills],
            'team_baron_kill_first': [team_baron_kill_first],
            'team_baron_kills': [team_baron_kills],
            'team_champion_kill_first': [team_champion_kill_first],
            'team_champion_kills': [team_champion_kills],
            'team_dragon_kill_first': [team_dragon_kill_first],
            'team_dragon_kills': [team_dragon_kills],
            'team_horde_kill_first': [team_horde_kill_first],
            'team_horde_kills': [team_horde_kills],
            'team_inhibitor_kill_first': [team_inhibitor_kill_first],
            'team_inhibitor_kills': [team_inhibitor_kills],
            'team_rift_herald_kill_first': [team_rift_herald_kill_first],
            'team_rift_herald_kills': [team_rift_herald_kills],
            'team_tower_kill_first': [team_tower_kill_first],
            'team_tower_kills': [team_tower_kills],
            'opposing_team_atakhan_kill_first': [opposing_team_atakhan_kill_first],
            'opposing_team_atakhan_kills': [opposing_team_atakhan_kills],
            'opposing_team_baron_kill_first': [opposing_team_baron_kill_first],
            'opposing_team_baron_kills': [opposing_team_baron_kills],
            'opposing_team_champion_kill_first': [opposing_team_champion_kill_first],
            'opposing_team_champion_kills': [opposing_team_champion_kills],
            'opposing_team_dragon_kill_first': [opposing_team_dragon_kill_first],
            'opposing_team_dragon_kills': [opposing_team_dragon_kills],
            'opposing_team_horde_kill_first': [opposing_team_horde_kill_first],
            'opposing_team_horde_kills': [opposing_team_horde_kills],
            'opposing_team_inhibitor_kill_first': [opposing_team_inhibitor_kill_first],
            'opposing_team_inhibitor_kills': [opposing_team_inhibitor_kills],
            'opposing_team_rift_herald_kill_first': [opposing_team_rift_herald_kill_first],
            'opposing_team_rift_herald_kills': [opposing_team_rift_herald_kills],
            'opposing_team_tower_kill_first': [opposing_team_tower_kill_first],
            'opposing_team_tower_kills': [opposing_team_tower_kills],
            'username': [username]
        }
    )
    return match_dataframe

In [ ]:
def retrieve_match_data(match_id_to_puuid_list: dict[str, list[str]], api_key:str):
    match_data_session = FuturesSession(executor=ProcessPoolExecutor(max_workers=10))
    retries = 5
    status_forcelist = [429]
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        respect_retry_after_header=True,
        status_forcelist=status_forcelist,
    )

    adapter = HTTPAdapter(max_retries=retry)

    match_data_session.mount('http://', adapter)
    match_data_session.mount('https://', adapter)

    dataframe_list = []
    t1 = time.time()
    for user_dict in tqdm(match_id_to_puuid_list):
        match_data_threads = [match_data_session.get(f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}') for match_id in user_dict['match_ids']]
        for future in tqdm(as_completed(match_data_threads)):
            resp = future.result()
            game = resp.json()
            puuid = user_dict['puuid']
            username = user_dict['username']
            match_dataframe = process_match_json(game, username, puuid)
            dataframe_list.append(match_dataframe)
    t2 = time.time()
    print(f'EXTRACTING MATCH DATA -- {round(t2 - t1, 2)}s')
    return dataframe_list

In [98]:
# TODO move this to a separate file
UHE_varsity_2025_2026 = [
  {
    'username': 'oanduh',
    'puuid': 'jgIFggFSUWMaiXjdREgaDpoQOcs5uyzwaS8tm781QGJtJHJazhaY1xPG5z9uUtSEy8CKDzQodTmTGQ'
  },
  {
    'username': 'bruhdun',
    'puuid': '_rpt4no7mbwiPc8L1-gacv4tvLjI0giGdBtKlc2ZCw8pN7BQjvzjbjw2ZlfZijiZMzAfRODlrN_KCQ'
  },
  {
    'username': 'hawnlilmonster',
    'puuid': 'ZzKy705E6fkzFEscD6HBqypYH3EWriEgumWJPgH-RU2l-NK-Zg40flY03DB-JxmuUWtlTGkzQYRqHw'
  },
  {
    'username': 'RoboBoto',
    'puuid': 'E-19GsPkwE-eUzeDs-R-CXe6QhA8Sykijb5jOib8It2_J1viqlVprlWOc4rT_2g6JnhHFeRY5tELrQ'
  },
  {
    'username': 'mazsu',
    'puuid': '1ZcFop5pM6PFXkveK0sJHoSIoJh53fHm-kF1MR3_zS1Rp7V71wA-u8y_8X6ivwb7P4jUuXUE8c6H7Q'
  }
]

api_key = os.getenv("riot_api_key")

In [101]:
UHE_varsity_2025_2026[0]
match_id_to_puuid_list = retrieve_user_match_ids([UHE_varsity_2025_2026[0]], api_key, start=100, count=3)

1it [00:02,  2.68s/it]

EXTRACTING MATCH IDS -- 2.68s


In [ ]:
# UHE_varsity_2025_2026 
# to isolate match selections just select specific indexes in the list
match_id_to_puuid_list = retrieve_user_match_ids(UHE_varsity_2025_2026, api_key, start=100, count=100)
dataframe_list = retrieve_match_data(match_id_to_puuid_list, api_key)
all_matches_dataframe = pd.concat(dataframe_list)


5it [00:02,  2.07it/s]


EXTRACTING MATCH IDS -- 2.42s


100it [02:00,  1.21s/it]:00<?, ?it/s]
100it [02:00,  1.21s/it]:01<08:05, 121.27s/it]
100it [02:01,  1.21s/it]:01<06:02, 120.88s/it]
100it [01:59,  1.20s/it]:02<04:01, 120.96s/it]
100it [02:01,  1.21s/it]:02<02:00, 120.59s/it]
100%|██████████| 5/5 [10:04<00:00, 120.82s/it]


EXTRACTING MATCH DATA -- 604.09s


In [88]:
def retrieve_perk_info():
    perks = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json').json()
    perkstyles = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json').json()
    
    perk_dict = {}
    for i in perkstyles['styles']:
        perk_dict[i['id']] = i['name']

    for i in perks:
        perk_dict[i['id']] = i['name']
    return perk_dict

In [89]:
perk_dict = retrieve_perk_info()
perk_columns = ['keystone', 'primary_style_selection_1', 'primary_style_selection_2', 
                'primary_style_selection_3', 'secondary_style_selection_1', 
                'secondary_style_selection_2', 'perk_defense', 'perk_flex', 'perk_offense']

all_matches_dataframe[perk_columns] = all_matches_dataframe[perk_columns].replace(perk_dict)

In [90]:
def retrieve_items_info():
    items = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json').json()
    
    items_dict = {}
    for i in items:
        items_dict[i['id']] = i['name']
    return items_dict

In [91]:
items_dict = retrieve_items_info()
item_columns = ['item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5', 'item_6']
all_matches_dataframe[item_columns] = all_matches_dataframe[item_columns].replace(items_dict)

In [92]:
all_matches_dataframe['uuid'] = all_matches_dataframe['match_id'] + '_' + all_matches_dataframe['puuid']
all_matches_dataframe = all_matches_dataframe.set_index('uuid')

In [127]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()
upsert(con=connection, df=all_matches_dataframe, schema='soloq', table_name='player_matches', create_schema=True, if_row_exists='update')

In [128]:
connection.commit()